We also need inflation rate, interest rate. Five factors.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [94]:
df=pd.read_csv('cny_daily.csv',index_col=0)
#all_three['DATE'] = pd.to_datetime(all_three['DATE'])
can=pd.read_csv('Canada_daily.csv',index_col=0)
jpy=pd.read_csv('jpy_daily.csv',index_col=0)
inflation=pd.read_csv('inflation_daily.csv',index_col=0)
interest=pd.read_csv('interest_daily.csv',index_col=0)

In [ ]:
all_three

In [96]:
interest

,DGS10
DATE,
2009-01-02,2.46
2009-01-05,2.49
2009-01-06,2.51
2009-01-07,2.52
2009-01-08,2.47
...,...
2019-12-25,.
2019-12-26,1.90
2019-12-27,1.88


In [97]:
inflation

,T10YIE
DATE,
2009-01-02,0.17
2009-01-05,0.15
2009-01-06,0.42
2009-01-07,0.47
2009-01-08,0.50
...,...
2019-12-25,.
2019-12-26,1.74
2019-12-27,1.73


In [98]:
df['jpy']=jpy['DEXJPUS']
df['can']=can['DEXCAUS']
df['cny']=all_three['DEXCHUS']
df['interest']=interest['DGS10']
df['inflation']=inflation['T10YIE']
df=df.drop(columns='DEXCHUS')
df

,jpy,can,cny,interest,inflation
DATE,,,,,
2009-01-02,91.1200,1.2101,6.8225,2.46,0.17
2009-01-05,93.2000,1.1920,6.8300,2.49,0.15
2009-01-06,94.2000,1.1822,6.8360,2.51,0.42
2009-01-07,92.7000,1.1850,6.8330,2.52,0.47
2009-01-08,91.1500,1.1890,6.8342,2.47,0.50
...,...,...,...,...,...
2019-12-25,.,.,.,.,.
2019-12-26,109.67,1.3124,6.9949,1.90,1.74
2019-12-27,109.47,1.3073,6.9954,1.88,1.73


In [99]:
df.to_csv('all_five.csv')

In [ ]:
################################################
# start at this step#

In [101]:
df=pd.read_csv('all_five.csv')
df

,jpy,can,cny,interest,inflation
0,91.12,1.2101,6.8225,2.46,0.17
1,93.2,1.192,6.83,2.49,0.15
2,94.2,1.1822,6.836,2.51,0.42
3,92.7,1.185,6.833,2.52,0.47
4,91.15,1.189,6.8342,2.47,0.5
...,...,...,...,...,...
2863,.,.,.,.,.
2864,109.67,1.3124,6.9949,1.9,1.74
2865,109.47,1.3073,6.9954,1.88,1.73
2866,108.85,1.3058,6.9864,1.9,1.75


In [102]:

df['jpy'] = np.where(df['jpy'] == '.', np.nan, df['jpy'])
df['can'] = np.where(df['can'] == '.', np.nan, df['can'])
df['cny'] = np.where(df['cny'] == '.', np.nan, df['cny'])
df['interest'] = np.where(df['interest'] == '.', np.nan, df['interest'])
df['inflation'] = np.where(df['inflation'] == '.', np.nan, df['inflation'])
df=df.dropna()
df


,jpy,can,cny,interest,inflation
0,91.12,1.2101,6.8225,2.46,0.17
1,93.2,1.192,6.83,2.49,0.15
2,94.2,1.1822,6.836,2.51,0.42
3,92.7,1.185,6.833,2.52,0.47
4,91.15,1.189,6.8342,2.47,0.5
...,...,...,...,...,...
2862,109.38,1.316,7.0064,1.9,1.75
2864,109.67,1.3124,6.9949,1.9,1.74
2865,109.47,1.3073,6.9954,1.88,1.73
2866,108.85,1.3058,6.9864,1.9,1.75


In [108]:
df.to_csv('all_five_dropna.csv')
df1=df

# The row are the Response (Y) and the columns are the predictor series (X). For grangercasuality test: #

In [112]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df1, variables = df.columns)   

,jpy_x,can_x,cny_x,interest_x,inflation_x
jpy_y,1.0000,0.3712,0.0574,0.0000,0.3290
can_y,0.0002,1.0000,0.0202,0.0491,0.0000
cny_y,0.0067,0.0000,1.0000,0.5164,0.0247
interest_y,0.4727,0.4292,0.2953,1.0000,0.0664
inflation_y,0.0000,0.0000,0.1509,0.0008,1.0000


# Cointegration test #

In [ ]:
import statsmodels.tsa.stattools as ts 



In [121]:
result=ts.coint(df1['jpy'], df1['can'])
result
    
    
    

(-2.9997867524668425,
 0.11025619232554884,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [122]:
result=ts.coint(df1['jpy'], df1['cny'])
result

(-1.2011473543166138,
 0.8576002630948625,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [125]:
result=ts.coint(df1['jpy'], df1['interest'])
result

(-1.4090350612768352,
 0.7943994951278808,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [129]:
result=ts.coint(df1['jpy'], df1['inflation'])
result

(-3.0459904169613186,
 0.09967402027273148,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [130]:
result=ts.coint(df1['can'], df1['cny'])
result

(-1.2702438260010671,
 0.8385349955327079,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [131]:
result=ts.coint(df1['can'], df1['interest'])
result

(-1.0768047003258907,
 0.8872855947795184,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [135]:
result=ts.coint(df1['can'], df1['inflation'])
result

(-3.4146695710096693,
 0.04074706042984389,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [133]:
result=ts.coint(df1['cny'], df1['interest'])
result

(-0.9627581546311769,
 0.9096858395199916,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [134]:
result=ts.coint(df1['cny'], df1['inflation'])
result

(-1.4117002886328598,
 0.7934745704532651,
 array([-3.90043713, -3.33835843, -3.04599655]))

In [136]:
result=ts.coint(df1['interest'], df1['inflation'])
result

(-1.8807657425123951,
 0.5897294708798675,
 array([-3.90043713, -3.33835843, -3.04599655]))

# Split the dataset #

In [137]:

nobs = 160
df_train, df_test = df1[0:-nobs], df1[-nobs:]

# Check size
print(df_train.shape)  # (2584,5)
print(df_test.shape)  # (160,5)
df_train.to_csv('df_train_ver1.csv')
df_test.to_csv('df_test_ver1.csv')


(2584, 5)
(160, 5)


In [ ]:
##################################Ignore the block above, use the data below.

In [138]:
df_train=pd.read_csv('df_train_ver1.csv')
df_test=pd.read_csv('df_test_ver1.csv')



# ADF test #

In [139]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.") 

In [140]:
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Unnamed: 0" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 1.2439
 No. Lags Chosen       = 24
 Critical value 1%     = -3.433
 Critical value 5%     = -2.863
 Critical value 10%    = -2.567
 => P-Value = 0.9963. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "jpy" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.0863
 No. Lags Chosen       = 3
 Critical value 1%     = -3.433
 Critical value 5%     = -2.863
 Critical value 10%    = -2.567
 => P-Value = 0.7205. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "can" 
    -----------------------------------------------
 Null Hypothesis: Data has uni

# log, then first difference #

In [142]:
# log once
df_train_log=df_train
df_train_log['can']=np.log(df_train['can'])
df_train_log['cny']=np.log(df_train['cny'])
df_train_log['jpy']=np.log(df_train['jpy'])
df_train_log['interest']=np.log(df_train['interest'])
df_train_log['inflation']=np.log(df_train['inflation'])

In [144]:
df_train_log = df_train_log.loc[:, ~df_train_log.columns.str.contains('^Unnamed')]
df_train_log

,jpy,can,cny,interest,inflation
0,4.512177,0.190703,1.920226,0.900161,-1.771957
1,4.534748,0.175633,1.921325,0.912283,-1.897120
2,4.545420,0.167377,1.922203,0.920283,-0.867501
3,4.529368,0.169743,1.921764,0.924259,-0.755023
4,4.512507,0.173113,1.921939,0.904218,-0.693147
...,...,...,...,...,...
2579,4.708268,0.296097,1.911052,0.920283,0.647103
2580,4.703567,0.298028,1.913313,0.896088,0.625938
2581,4.701571,0.296989,1.914243,0.912283,0.625938
2582,4.696381,0.299364,1.920680,0.896088,0.615186


In [188]:
df_train_diff.to_csv('df_train_diff.csv')

In [ ]:
#then 1st difference

In [146]:
df_train_diff=df_train_log
df_train_diff['jpy']=df_train_diff['jpy'].diff()
df_train_diff['can']=df_train_diff['can'].diff()
df_train_diff['cny']=df_train_diff['cny'].diff()
df_train_diff['inflation']=df_train_diff['inflation'].diff()
df_train_diff['interest']=df_train_diff['interest'].diff()
df_train_diff=df_train_diff.dropna()
df_train_diff

<ipython-input-146-e396f6e0eee1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_diff['jpy']=df_train_diff['jpy'].diff()
<ipython-input-146-e396f6e0eee1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_diff['can']=df_train_diff['can'].diff()
<ipython-input-146-e396f6e0eee1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

,jpy,can,cny,interest,inflation
2,-0.011898,0.006815,-0.000221,-0.004121,1.154783
3,-0.026724,0.010621,-0.001317,-0.004024,-0.917141
4,-0.000810,0.001004,0.000615,-0.024017,-0.050603
5,0.006939,-0.000850,-0.000059,0.003714,0.051453
6,-0.004924,0.010400,0.000059,-0.021413,-0.061143
...,...,...,...,...,...
2579,-0.002793,0.004761,0.003928,-0.007952,-0.010526
2580,-0.000920,0.000369,-0.001667,-0.012313,-0.021165
2581,0.002706,-0.002972,-0.001331,0.040389,0.021165
2582,-0.003194,0.003415,0.005508,-0.032389,-0.010753


# After we do the log and then the difference, do the ADF test again: #

In [147]:
for name, column in df_train_diff.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "jpy" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -16.4398
 No. Lags Chosen       = 28
 Critical value 1%     = -3.433
 Critical value 5%     = -2.863
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "can" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -16.446
 No. Lags Chosen       = 28
 Critical value 1%     = -3.433
 Critical value 5%     = -2.863
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "cny" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05


# passed the adf test now. then... #

In [151]:
model = VAR(df_train_diff)
for i in [1,2,3,4,5,6,7,8,9,10]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  -46.988546977303024
BIC :  -46.92048112453605
FPE :  3.918621827814305e-21
HQIC:  -46.963875866711014 

Lag Order = 2
AIC :  -47.901792550203034
BIC :  -47.77696504737925
FPE :  1.5722294768894104e-21
HQIC:  -47.856546752840835 

Lag Order = 3
AIC :  -48.20477985033961
BIC :  -48.02315419680316
FPE :  1.1612626430107023e-21
HQIC:  -48.13894532473894 

Lag Order = 4
AIC :  -48.38217045866783
BIC :  -48.14371011566263
FPE :  9.725034032608086e-22
HQIC:  -48.29573314827628 

Lag Order = 5
AIC :  -48.55135520013266
BIC :  -48.256023590747795
FPE :  8.211374024512624e-22
HQIC:  -48.444301033291694 

Lag Order = 6
AIC :  -48.735665298945726
BIC :  -48.38342580806073
FPE :  6.829234285032255e-22
HQIC:  -48.60798018886862 

Lag Order = 7
AIC :  -48.964912269648366
BIC :  -48.55572824387842
FPE :  5.430161555786803e-22
HQIC:  -48.8165821143981 

Lag Order = 8
AIC :  -49.010286874707404
BIC :  -48.544121622348435
FPE :  5.189299123220893e-22
HQIC:  -48.84129755717452 

Lag O

/home/zcw/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


# Train the VAR model: #
R could product a much more better version:
Please see: model_parameters.txt

In [152]:
model_fitted = model.fit(1)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 30, Oct, 2020
Time:                     01:28:57
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                   -46.9205
Nobs:                     2581.00    HQIC:                  -46.9639
Log likelihood:           42357.3    FPE:                3.91862e-21
AIC:                     -46.9885    Det(Omega_mle):     3.87339e-21
--------------------------------------------------------------------
Results for equation jpy
                  coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------
const               -0.000008         0.000150           -0.056           0.955
L1.jpy              -0.512672         0.018318          -27.988           0.000
L1.can               0.034337         0.019440            1.766   

In [156]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

out

array([2.36229064, 2.33867165, 2.37876598, 2.35461925, 1.88509623])

In [189]:


# Input data for forecasting
forecast_input = df_train_diff.values[-1:]
forecast_input

array([[ 0.00528089, -0.00698154, -0.00700924,  0.02432481,  0.02683893]])

In [190]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=df.columns + '_2d')
df_forecast

,jpy_2d,can_2d,cny_2d,interest_2d,inflation_2d
2701,-0.001536,0.003559,3.606978e-03,-0.012727,-0.010036
2702,0.000099,-0.001838,-1.851077e-03,0.006412,0.003185
2703,0.000318,0.000976,9.470756e-04,-0.003117,-0.001468
2704,-0.000360,-0.000510,-4.797649e-04,0.001430,0.000166
2705,0.000275,0.000281,2.424137e-04,-0.000631,-0.000428
...,...,...,...,...,...
2862,-0.000003,0.000005,6.550518e-07,-0.000007,-0.000263
2864,-0.000003,0.000005,6.550518e-07,-0.000007,-0.000263
2865,-0.000003,0.000005,6.550518e-07,-0.000007,-0.000263
2866,-0.000003,0.000005,6.550518e-07,-0.000007,-0.000263


In [191]:
df_forecast=df_forecast.cumsum()
df_forecast


,jpy_2d,can_2d,cny_2d,interest_2d,inflation_2d
2701,-0.001536,0.003559,0.003607,-0.012727,-0.010036
2702,-0.001437,0.001721,0.001756,-0.006315,-0.006850
2703,-0.001119,0.002697,0.002703,-0.009433,-0.008318
2704,-0.001478,0.002186,0.002223,-0.008002,-0.008152
2705,-0.001204,0.002468,0.002466,-0.008634,-0.008580
...,...,...,...,...,...
2862,-0.001731,0.003190,0.002484,-0.009471,-0.048288
2864,-0.001734,0.003196,0.002485,-0.009478,-0.048551
2865,-0.001737,0.003201,0.002485,-0.009485,-0.048814
2866,-0.001739,0.003207,0.002486,-0.009491,-0.049078


In [ ]:
#2583 	4.696472 	0.294757 	1.920109 	0.904218 	0.631272

In [192]:
df_forecast['jpy_2d']=df_forecast['jpy_2d']+4.696472
df_forecast['can_2d']=df_forecast['can_2d']+0.294757
df_forecast['cny_2d']=df_forecast['cny_2d']+1.920109
df_forecast['interest_2d']=df_forecast['interest_2d']+0.904218
df_forecast['inflation_2d']=df_forecast['inflation_2d']+0.631272
df_forecast

,jpy_2d,can_2d,cny_2d,interest_2d,inflation_2d
2701,4.694936,0.298316,1.923716,0.891491,0.621236
2702,4.695035,0.296478,1.921865,0.897903,0.624422
2703,4.695353,0.297454,1.922812,0.894785,0.622954
2704,4.694994,0.296943,1.922332,0.896216,0.623120
2705,4.695268,0.297225,1.922575,0.895584,0.622692
...,...,...,...,...,...
2862,4.694741,0.297947,1.922593,0.894747,0.582984
2864,4.694738,0.297953,1.922594,0.894740,0.582721
2865,4.694735,0.297958,1.922594,0.894733,0.582458
2866,4.694733,0.297964,1.922595,0.894727,0.582194


In [193]:
df_forecast=np.exp(df_forecast)

In [194]:
df_forecast

,jpy_2d,can_2d,cny_2d,interest_2d,inflation_2d
2701,109.391861,1.347588,6.846352,2.438763,1.861228
2702,109.402675,1.345113,6.833691,2.454450,1.867166
2703,109.437458,1.346426,6.840166,2.446810,1.864427
2704,109.398122,1.345739,6.836885,2.450313,1.864736
2705,109.428173,1.346118,6.838543,2.448766,1.863938
...,...,...,...,...,...
2862,109.370451,1.347091,6.838668,2.446717,1.791376
2864,109.370151,1.347098,6.838672,2.446700,1.790905
2865,109.369852,1.347105,6.838677,2.446684,1.790433
2866,109.369552,1.347113,6.838681,2.446667,1.789962


In [186]:
df_forecast.to_csv('df_final_forecast.csv')

https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/